In [ ]:
#Установка библиотеки transformers и др.для  GC

import sys
ENV_COLAB = 'google.colab' in sys.modules

if ENV_COLAB:
    !pip install transformers
    import transformers as ppb

    print('Environment: Google Colab')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 9.4 MB/s eta 0:00:00
Environment: Google Colab


In [ ]:
! pip install tensorflow-gpu==2.1.0

%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.1.0 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.12.0)
ERROR: No matching distribution found for tensorflow-gpu==2.1.0
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


In [ ]:
import numpy as np
import pandas as pd
import torch
from tqdm import notebook
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
#from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import random
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import time

import sys
sys.setrecursionlimit(2000)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

# делаем выборку 1000 случайных объектов
corpus_bert =data.sample(1000).reset_index(drop=True)
corpus_bert.shape

corpus_bert.toxic.value_counts()

0    918
1     82
Name: toxic, dtype: int64

In [ ]:
#Загружаю предобученную модель Bert
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'unitary/toxic-bert')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#Токенизирую тексты - то есть каждый комментарий станет списком токенов:
tokenized = corpus_bert['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True))
#tokenized = batch_1.text.apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True)))
#Чтобы в модель загрузить сразу все данные, надо выровнять длину комментариев, чтобы в BERT подать двумерный массив:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

np.array(padded).shape

(1000, 512)

In [ ]:
#Создаю маску, чтобы игнорировать паддинг:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 512)

In [ ]:
# проверим, есть ли в системе видеоускоритель

!nvidia-smi

Sat Mar 25 20:54:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    28W /  70W |    339MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# создадим переменную, в которой будет указан девайс, на котором будет проводить вычисления
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# отправим модель на gpu
model = model.to(device)

In [ ]:
#делаем эмбендинг
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch = batch.to(device) # батч на видеокарту
        attention_mask_batch = attention_mask_batch.to(device) # маску на видеокарту
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
#создаем признаки
features = np.concatenate(embeddings)
features.shape
# обучим и протестируйем модель
# разделим наши данные на признаки features и целевую переменную target
target = corpus_bert['toxic']
# разделяем на обучающую и валидационную выборку в пропорции 80/20
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2,
                                                    random_state=12345)

In [ ]:
#Создадим функцию для кросс-валидации
def cross_val_model(model, features, target, name_model):#silent
    #if silent:
    #    features_train_resample = count_tf_idf.transform(features['text'])
    model = model
    f1_result = cross_val_score(model, features, target,
                           cv=5, scoring='f1').mean()

    print(f'На {name_model} F1 = {round(f1_result,4)}')
    return f1_result

### **LogisticRegression**

In [ ]:
%%time
#обучаем модель на логистической регрессии
model_lr = LogisticRegression(random_state=12345)
model_lr.fit(features_train, target_train,)

start = time.time()
f1_lr_cv = cross_val_model(model_lr, features_train, target_train,\
                              'LogisticRegression') #, silent=False

end = time.time()
t_lr_cv=round(end-start,2)

На LogisticRegression F1 = 0.9217
CPU times: user 621 ms, sys: 307 ms, total: 929 ms
Wall time: 1.65 s


In [ ]:
f1_test_lr = f1_score(target_test, model_lr.predict(features_test))
print(f'LogisticRegression F1 = {round(f1_test_lr,4)}')

LogisticRegression F1 = 0.9778


### **SGDClassifier**

In [ ]:
%%time
#обучаем модель на SGDClassifier
model_sgdc = SGDClassifier(random_state=12345, class_weight='balanced')
model_sgdc.fit(features_train, target_train,)

start = time.time()
f1_sgdc_cv = cross_val_model(model_sgdc, features_train, target_train,\
                              'SGDClassifier') #, silent=False

end = time.time()
t_sgdc_cv=round(end-start,2)

На SGDClassifier F1 = 0.8623
CPU times: user 161 ms, sys: 76.9 ms, total: 238 ms
Wall time: 185 ms


In [ ]:
#f1_test_sgdc = f1_score(target_test, model_sgdc.predict(features_test))
#print(f'SGDClassifier F1 = {round(f1_test_sgdc,4)}')

### **RandomForestClassifier**

In [ ]:
%%time
model_rfc = RandomForestClassifier(random_state=12345)
model_rfc.fit(features_train, target_train,)

start = time.time()
f1_rfc_cv = cross_val_model(model_rfc, features_train, target_train,\
                              'RandomForestClassifier') #, silent=False

end = time.time()
t_rfc_cv=round(end-start,2)

На RandomForestClassifier F1 = 0.9152
CPU times: user 3.02 s, sys: 11 ms, total: 3.03 s
Wall time: 3.02 s


In [ ]:
#f1_test_rf = f1_score(target_test, model_rfc.predict(features_test))
#print(f'RandomForestClassifier F1 = {round(f1_test_rf, 4)}')

### **DecisionTreeClassifier**

In [ ]:
%%time
model_dtc = DecisionTreeClassifier(random_state=12345, class_weight='balanced')
model_dtc.fit(features_train, target_train)

start = time.time()
f1_rfc_cv = cross_val_model(model_rfc, features_train, target_train,\
                              'DecisionTreeClassifier') #, silent=False

end = time.time()
t__dtc_cv=round(end-start,2)

На DecisionTreeClassifier F1 = 0.9152
CPU times: user 3.28 s, sys: 6.36 ms, total: 3.29 s
Wall time: 3.37 s


In [ ]:
#f1_test_dtc = f1_score(target_test, model_dtc.predict(features_test))
#print(f'DecisionTreeClassifier F1 = {round(f1_test_dtc, 4)}')

In [ ]:
def result_write(model_name,  f1_cv, f1 ):  #t,f1_p,,time_fit,
#функция добавляет значения метрики f1 на БЕРТ с использованием разных моделей

  result_corpus.loc[model_name,'F1_final_BERT'] = f1
  result_corpus.loc[model_name,'F1_cv_BERT'] = f1_cv
  #result_corpus.loc[model_name,'Time_fit'] = time_fit
  return result_corpus

result_corpus = pd.DataFrame(columns=[ 'F1_cv_BERT', 'F1_final_BERT']) #'Treshold',,'F1_predict_proba','Time_fit','F1_predict_BERT',
result_corpus

,F1_cv_BERT,F1_final_BERT


In [ ]:
result_write('LogisticRegression', round(f1_lr_cv,4), round(f1_test_lr,4))
result_write('SGDClassifier', round(f1_sgdc_cv,4), 'None')
result_write('RandomForestClassifier', round(f1_rfc_cv, 4), 'None')
result_write('DecisionTreeClassifier', round( f1_rfc_cv, 4), 'None')

,F1_cv_BERT,F1_final_BERT
LogisticRegression,0.9217,0.9778
SGDClassifier,0.8623,None
RandomForestClassifier,0.9152,None
DecisionTreeClassifier,0.9152,None
